# Random Access Lists
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/matyama/pfds/blob/main/notebooks/rlists.ipynb)

A *Random Access List*, also known as a *one-sided flexible array*, is a list-like data structure that supports two additional operations:
 - `lookup` retrieves an item given its position (index) in the list
 - `update` stores new item at given position in the list

### Overview
|                  instance                  | amortization | empty | isEmpty |        cons       |        head        |        tail       |   lookup  |   update  |
|:------------------------------------------:|:------------:|:-----:|:-------:|:-----------------:|:------------------:|:-----------------:|:---------:|:---------:|
|          Binary Random Access List         |      no      |  O(1) |   O(1)  |     O(log(n))     | O(log(n)) / O(1)** |     O(log(n))     | O(log(n)) | O(log(n)) |
|     Zeroless Binary Random Access List     |      no      |  O(1) |   O(1)  |     O(log(n))     |        O(1)        |     O(log(n))     | O(log(i)) | O(log(i)) |
| Zeroless Redundant Bin. Random Access List |      yes     |  O(1) |   O(1)  | O(log(n)) / O(1)* |        O(1)        | O(log(n)) / O(1)* | O(log(i)) | O(log(i)) |

where $n$ is the list size and $i$ is the index parameter of the `lookup`/`update`.

** amortized time*

*** with explicit reference to the head element*

### Constant-time `head`
One can trivially make the `head` of any random access list constant time by either
 - Explicitly tracking the head element in addition to the list itself (similarly to `ExplicitMinHeap` class - see the `ExplicitHeadList` below)
 - Using a *sparse binary representation* and a *binomial tree* or a *pennant* representation then the head of the list would be stored in the root of the first tree (and thus accessible in $O(1)$ time)

In [ ]:
class RandomAccessList l where
    
    -- | Construct new (empty) list
    empty :: l a
    
    -- | Check whether a list is empty
    isEmpty :: l a -> Bool
    
    -- | Prepend new item to the front a list
    cons :: a -> l a -> l a
    
    -- | Retrieve the head item of a non-empty list
    head :: l a -> a
    
    -- | Remove the head of a non-empty list and retrieve the rear
    tail :: l a -> l a
    
    -- | Retrieve a list item at given index
    lookup :: Int -> l a -> a
    
    -- | Store an item at given position in a list
    update :: Int -> a -> l a -> l a


data ExplicitHeadList l a = Empty | EH a (l a)

instance RandomAccessList l => RandomAccessList (ExplicitHeadList l) where
    
    empty = Empty
    
    isEmpty Empty = True
    isEmpty _ = False
    
    cons x Empty = EH x (cons x empty)
    cons x (EH _ l) = EH x (cons x l)
    
    -- | Retrieve the head item of a non-empty list in O(1) worst case time
    head Empty = error "List is empty"
    head (EH x _) = x
    
    -- | Remove the head of a non-empty list and retrieve the rear in O(max(head, tail)) of
    -- | the underlying list.
    -- |
    -- | This implementaiton assumes that the bound on 'head' is no worse than that of the 'tail'.
    -- | If this is not true for the wrapped 'RandomAccessList' than the performance of 'tail' will degrade.
    tail Empty = error "List is empty"
    tail (EH _ l) = let l' = tail l in EH (head l') l'
    
    lookup _ Empty = error "List is empty"
    lookup i (EH _ l) = lookup i l
    
    update _ _ Empty = error "List is empty"
    update 0 y (EH _ l) = EH y (update 0 y l)
    update i y (EH x l) = EH x (update i y l)

## Binary Random Access List

### Representation
`BinaryList` presented below is based on a (dense) binary numerical representation - that is, a `BinaryList` of with $n$ elements constins a *complete binary leaf tree* for each one in the dense binary representation of $n$.

The random access list is represented as a list of `Digit`s which can ither be `Zero` or `One (Tree a)` where the tree is a complete binary leaf tree (i.e. a binary tree in with data stored just in leafs).

In addition to storing list elements in the `Leaf`s, the implementation below also includes the tree size in each `Node`. Note that the size is redundant since it can be deduced either from the parent size or by the position in the list of digits (which is kept in increasing order of the size - i.e. rank).

Finally, we store the elements in a left-to-right order both within and between trees. Under this repreentation both the number of trees and the tree depth is logarithmic in $n$.

### Operations
The `cons` and `tail` operations (resp. `consTree` and `unconsTree`) are analogous to `inc` and `dec` respective implementation of `DenseNat`. Since they do at most $O(1)$ work per `Digit` and there's at most `O(log(n))` threes in the list, their worst case time is `O(log(n))`. And since `head` is basically the same as `tail` but instead of discarding the head it resurns it, it also runs in `O(log(n))` time.

The other two operations, that is `lookup` and `update` both take the advantage of the list represenation described above. They both perform two successive logarithmic steps and thus run in `O(log(n))` worst case time:
 1. find the appropriate tree in the list by comparing its size and the index
 1. in this tree find and return (update) by again checking against the size field whether to search the left or right sub-tree

The only difference between these two is that `update` also copies the path from the root to the updated leaf.

In [ ]:
-- | Complete Binary Leaf Tree: Leaf (element) or Node (weight/size, left sub-tree, right sub-tree)
data Tree a = Leaf a | Node Int (Tree a) (Tree a)

data Digit a = Zero | One (Tree a)

-- | BRL = dense representation of the size of the list where each one is a complete binary tree
-- |
-- | Note: Trees are stored in an increasing order of their size and the order of elements both
-- |       within and between trees is left to right.
newtype BinaryList a = BL [Digit a]

-- | Retrieve the size of a binary tree in O(1) worst case time
size :: Tree a -> Int
size (Leaf _) = 1
size (Node w _ _) = w

-- | Create new node with given trees as respective left and right sub-trees in O(1) worst case time.
link :: Tree a -> Tree a -> Tree a
link t1 t2 = Node (size t1 + size t2) t1 t2

-- | Add a tree to a binary-represented list.
-- |
-- | This is an analogy of incrementing a binary number in a dense representation and
-- | since there's at most logarithmic number of digits and 'link' takes constant time, this
-- | function runs in O(log(n)) worst case time.
consTree :: Tree a -> [Digit a] -> [Digit a]
consTree t [] = [One t]
consTree t (Zero:ts) = One t : ts
consTree t1 ((One t2):ts) = Zero : consTree (link t1 t2) ts

-- | Extract a tree from a binary-represented list and return it alongside new list.
-- |
-- | This is an analogy of decrementing a binary number in a dense representation and
-- | since there's at most logarithmic number of digits, this function runs in O(log(n)) worst case time.
unconsTree :: [Digit a] -> (Tree a, [Digit a])
unconsTree [] = error "List is empty"
unconsTree [One t] = (t, [])
unconsTree ((One t):ts) = (t, Zero:ts)
unconsTree (Zero:ts) = let (Node _ t1 t2, ts') = unconsTree ts in (t1, One t2 : ts')

instance RandomAccessList BinaryList where
    
    -- | Construct new (empty) list in O(1) worst case time
    empty = BL []
    
    -- | Check whether a list is empty in O(1) worst case time
    isEmpty (BL ts) = null ts
    
    -- | Prepend new item to the front a list in O(log(n)) worst case time.
    cons x (BL ts) = BL $ consTree (Leaf x) ts
    
    -- | Retrieve the head item of a non-empty list in O(log(n)) worst case time.
    head (BL ts) = let (Leaf x, _) = unconsTree ts in x
    
    -- | Remove the head of a non-empty list and retrieve the rear in O(log(n)) worst case time.
    tail (BL ts) = let (_, ts') = unconsTree ts in BL ts'
    
    -- | Retrieve a list item at given index in O(log(n)) worst case time.
    -- |
    -- | Lookup performs two steps - each running in logarithmic time:
    -- |  1. 'lookup'' will find an appropriate tree in the list of digits 'ts'
    -- |  1. 'lookupTree' will search this tree for the leaf element
    -- |
    -- | Note: The second case of the lookup will not loop forever as it will ither fail for
    -- |       an empty list or there will be at least one tree represented by a 'One'.
    lookup i (BL ts) = lookup' i ts
        where
            lookup' _ [] = error "List is empty"
            lookup' i (Zero:ts) = lookup' i ts
            lookup' i ((One t):ts) =
                if i < size t then lookupTree i t
                else lookup' (i - size t) ts
            --
            lookupTree 0 (Leaf x) = x
            lookupTree _ (Leaf _) = error "Index out of bounds"
            lookupTree i (Node w t1 t2) =
                if i < w `div` 2 then lookupTree i t1
                else lookupTree (i - w `div` 2) t2
    
    -- | Store an item at given position in a list in O(log(n)) worst case time
    -- |
    -- | Update does similar steps to the 'lookup':
    -- |  1. 'update'' will find an appropriate tree in the list of digits 'ts'
    -- |  1. 'updateTree' will search this tree for the leaf element and update it
    -- |
    -- | Note: The second case of the update helper cannot loop forever by the same argument
    -- |       explaind in 'lookup'.
    update i y (BL ts) = BL (update' i y ts)
        where
            update' _ _ [] = error "List is empty"
            update' i y (Zero:ts) = update' i y ts
            update' i y ((One t):ts) =
                if i < size t then One (updateTree i y t) : ts
                else One t : update' (i - size t) y ts
            --
            updateTree 0 y (Leaf x) = Leaf y
            updateTree _ _ (Leaf _) = error "Index out of bounds"
            updateTree i y (Node w t1 t2) =
                if i < w `div` 2 then Node w (updateTree i y t1) t2
                else Node w t1 (updateTree (i - w `div` 2) y t2)

## Zeroless Binary Random Access List

### Representation
A `ZerolessBinaryList` is an adaptation of the `BinaryList` which uses the *(dense) zeroless binary numerical representation* in which each `Digit` is either
 - `One` containing single complete binary tree
 - or `Two` holding two complete binary trees

### Operations
Since in this representation there's aways some tree in the initial digit, `head` can be implemented in $O(1)$ worst case time. Moreover, `lookup` and `update` can skip a whole tree for `Two` if the index $i$ is low enough which improves the complexity to $O(log(i))$.

The `consTree` and `unconsTree` are an analogue of the `inc` and `dec` operations of the `ZerolessNat`. Other list operations are adapted from `BinaryList` to work on the above representation.

In [ ]:
data Digit a = One (Tree a) | Two (Tree a) (Tree a)

-- | ZBL = zeroless representation of the size of the list where each one/two is one/two complete binary tree(s)
newtype ZerolessBinaryList a = ZBL [Digit a]

-- | Add a tree to a zeroless binary-represented list.
-- |
-- | Note: This implementation is just an adaptation of 'consTree' for the 'BinaryList'
-- |       that is based on the 'inc' for 'ZerolessNat'.
consTree :: Tree a -> [Digit a] -> [Digit a]
consTree t [] = [One t]
consTree t1 ((One t2):ts) = Two t1 t2 : ts
consTree t ((Two t1 t2):ts) = One t : consTree (link t1 t2) ts

-- | Extract a tree from a zeroless binary-represented list and return it alongside new list.
-- |
-- | Note: This implementation is just an adaptation of 'unconsTree' for the 'BinaryList'
-- |       that is based on the 'dec' for 'ZerolessNat'.
unconsTree :: [Digit a] -> (Tree a, [Digit a])
unconsTree [] = error "List is empty"
unconsTree [One t] = (t, [])
unconsTree ((Two t1 t2):ts) = (t1, One t2 : ts)
unconsTree ((One t):ts) = let (Node _ t1 t2, ts') = unconsTree ts in (t, Two t1 t2 : ts')

instance RandomAccessList ZerolessBinaryList where
    
    -- | Construct new (empty) list in O(1) worst case time
    empty = ZBL []
    
    -- | Check whether a list is empty in O(1) worst case time
    isEmpty (ZBL ts) = null ts
    
    -- | Prepend new item to the front a list in O(log(n)) worst case time.
    cons x (ZBL ts) = ZBL $ consTree (Leaf x) ts
    
    -- | Retrieve the head item of a non-empty list in O(1) worst case time.
    -- |
    -- | Because of the zeroless representation, there's always a tree in the first digit position
    -- | which is accessible in constant time and thus the call to 'unconsTree' is no longer needed here.
    head (ZBL ((One (Leaf x)):_)) = x
    head (ZBL ((Two (Leaf x) (Leaf _)):_)) = x
    
    -- | Remove the head of a non-empty list and retrieve the rear in O(log(n)) worst case time.
    tail (ZBL ts) = let (_, ts') = unconsTree ts in ZBL ts'
    
    -- | Retrieve a list item at given index in O(log(i)) worst case time.
    -- |
    -- | Note: Because the zeroless representation uses `Two`, the 'lookup'' helper can skip
    -- |       the second sub-tree if the index 'i' is low. Therefore the complexity is O(log(i))
    -- |       instead of O(log(n)).
    lookup i (ZBL ts) = lookup' i ts
        where
            lookup' _ [] = error "List is empty"
            lookup' i ((One t):ts)
                | i < size t = lookupTree i t
                | otherwise = lookup' (i - size t) ts
            lookup' i ((Two t1 t2):ts)
                | i < size t1 = lookupTree i t1
                | i - size t1 < size t2 = lookupTree (i - size t1) t2
                | otherwise = lookup' (i - size t1 - size t2) ts
            --
            lookupTree 0 (Leaf x) = x
            lookupTree _ (Leaf _) = error "Index out of bounds"
            lookupTree i (Node w t1 t2) =
                if i < w `div` 2 then lookupTree i t1
                else lookupTree (i - w `div` 2) t2
    
    -- | Store an item at given position in a list in O(log(i)) worst case time.
    -- |
    -- | Note: Implementation is similar to 'lookup' but 'update' additionally
    -- |       accumulates path from the root to build the updated tree and list.
    update i y (ZBL ts) = ZBL (update' i y ts)
        where
            update' _ _ [] = error "List is empty"
            update' i y ((One t):ts)
                | i < size t = One (updateTree i y t) : ts
                | otherwise = One t : update' (i - size t) y ts
            update' i y (tt @ (Two t1 t2):ts)
                | i < size t1 = Two (updateTree i y t1) t2 : ts
                | i - size t1 < size t2 = Two t1 (updateTree (i - size t1) y t2) : ts
                | otherwise = tt : update' (i - size t1 - size t2) y ts
            --
            updateTree 0 y (Leaf x) = Leaf y
            updateTree _ _ (Leaf _) = error "Index out of bounds"
            updateTree i y (Node w t1 t2) =
                if i < w `div` 2 then Node w (updateTree i y t1) t2
                else Node w t1 (updateTree (i - w `div` 2) y t2)

## Zeroless Redundant Binary Random Access List

### Representation
A `RedundantBinaryList` is an adaptation of the `ZerolessBinaryList` which adds a *redundant numerical representation* on top of the *(dense) zeroless binary numerical representation* in which each `Digit` is either
 - `One` containing single complete binary tree (the "dangerous" state for `consTree`)
 - or `Two` holding two complete binary trees (the "safe" state)
 - or `Three` holding three complete binary trees (the "dangerous" state for `unconsTree`)

### Operations
The redundancy is motivated by the `LazyNat` instance and makes the `consTree` (analogous to `inc`) and `unconsTree` (analogous to `dec`) to run in $O(1)$ amortized time. Due to this fact, `cons` and `tail` also run in this improved amortized complexity and `head` is still worst case $O(1)$ due to the zeroless representation.

The `One` and `Three` digits are the "dangerous" states for respective operations while `Two` is the "safe" one.

### Extensions
Finally, as it's common to other lazy amortized structures, *scheduling* can improve the amortized bounds on `cons` and `tail` to worst case bounds - meaning to $O(1)$ worst case time. Introduction and management of a schedule can be achieved in a way similar to the represenation and operations of the `ScheduledBinomialHeap`.

In [ ]:
data Digit a = One (Tree a) | Two (Tree a) (Tree a) | Three (Tree a) (Tree a) (Tree a)

-- | RBL = zeroless redundant representation of the size of the list where
-- |       each one/two/three is one/two/three complete binary tree(s)
newtype RedundantBinaryList a = RBL [Digit a]

-- | Add a tree to a zeroless redundant binary-represented list in O(1) amortized time.
-- |
-- | Note: This implementation is just an adaptation of 'consTree' for the 'ZerolessBinaryList'
-- |       that is based on the 'inc' for 'LazyNat'.
consTree :: Tree a -> [Digit a] -> [Digit a]
consTree t [] = [One t]
consTree t1 ((One t2):ts) = Two t1 t2 : ts
consTree t1 ((Two t2 t3):ts) = Three t1 t2 t3 : ts
consTree t ((Three t1 t2 t3):ts) = Two t t1 : consTree (link t2 t3) ts

-- | Extract a tree from a zeroless redundantly binary-represented list and
-- | return it alongside new list in O(1) amortized time.
-- |
-- | Note: This implementation is just an adaptation of 'unconsTree' for the 'ZerolessBinaryList'
-- |       that is based on the 'dec' for 'LazyNat'.
unconsTree :: [Digit a] -> (Tree a, [Digit a])
unconsTree [] = error "List is empty"
unconsTree [One t] = (t, [])
unconsTree ((Two t1 t2):ts) = (t1, One t2 : ts)
unconsTree ((Three t1 t2 t3):ts) = (t1, Two t2 t3 : ts)
unconsTree ((One t):ts) = let (Node _ t1 t2, ts') = unconsTree ts in (t, Two t1 t2 : ts')

-- | Note: This implementation is just an adaptation of the 'ZerolessBinaryList' instance to
-- |       the redundant representation of 'Digit' - i.e. adding cases for 'Three'.
instance RandomAccessList RedundantBinaryList where
    
    -- | Construct new (empty) list in O(1) worst case time
    empty = RBL []
    
    -- | Check whether a list is empty in O(1) worst case time
    isEmpty (RBL ts) = null ts
    
    -- | Prepend new item to the front a list in O(log(n)) worst case and O(1) amortized time.
    cons x (RBL ts) = RBL $ consTree (Leaf x) ts
    
    -- | Retrieve the head item of a non-empty list in O(1) worst case time.
    head (RBL ((One (Leaf x)):_)) = x
    head (RBL ((Two (Leaf x) (Leaf _)):_)) = x
    head (RBL ((Three (Leaf x) (Leaf _) (Leaf _)):_)) = x
    
    -- | Remove the head of a non-empty list and retrieve the rear in O(log(n)) worst case and O(1) amortized time.
    tail (RBL ts) = let (_, ts') = unconsTree ts in RBL ts'
    
    -- | Retrieve a list item at given index in O(log(i)) worst case time.
    lookup i (RBL ts) = lookup' i ts
        where
            lookup' _ [] = error "List is empty"
            lookup' i ((One t):ts)
                | i < size t = lookupTree i t
                | otherwise = lookup' (i - size t) ts
            lookup' i ((Two t1 t2):ts)
                | i < size t1 = lookupTree i t1
                | i - size t1 < size t2 = lookupTree (i - size t1) t2
                | otherwise = lookup' (i - size t1 - size t2) ts
            lookup' i ((Three t1 t2 t3):ts)
                | i < size t1 = lookupTree i t1
                | i - size t1 < size t2 = lookupTree (i - size t1) t2
                | i - size t1 - size t2 < size t3 = lookupTree (i - size t1 - size t2) t2
                | otherwise = lookup' (i - size t1 - size t2 - size t3) ts
            --
            lookupTree 0 (Leaf x) = x
            lookupTree _ (Leaf _) = error "Index out of bounds"
            lookupTree i (Node w t1 t2) =
                if i < w `div` 2 then lookupTree i t1
                else lookupTree (i - w `div` 2) t2
    
    -- | Store an item at given position in a list in O(log(i)) worst case time.
    update i y (RBL ts) = RBL (update' i y ts)
        where
            update' _ _ [] = error "List is empty"
            update' i y ((One t):ts)
                | i < size t = One (updateTree i y t) : ts
                | otherwise = One t : update' (i - size t) y ts
            update' i y (tt @ (Two t1 t2):ts)
                | i < size t1 = Two (updateTree i y t1) t2 : ts
                | i - size t1 < size t2 = Two t1 (updateTree (i - size t1) y t2) : ts
                | otherwise = tt : update' (i - size t1 - size t2) y ts
            update' i y (ttt @ (Three t1 t2 t3):ts)
                | i < size t1 = Three (updateTree i y t1) t2 t3 : ts
                | i - size t1 < size t2 = Three t1 (updateTree (i - size t1) y t2) t3 : ts
                | i - size t1 - size t2 < size t3 = Three t1 t2 (updateTree (i - size t1 - size t2) y t3) : ts
                | otherwise = ttt : update' (i - size t1 - size t2 - size t3) y ts
            --
            updateTree 0 y (Leaf x) = Leaf y
            updateTree _ _ (Leaf _) = error "Index out of bounds"
            updateTree i y (Node w t1 t2) =
                if i < w `div` 2 then Node w (updateTree i y t1) t2
                else Node w t1 (updateTree (i - w `div` 2) y t2)